In [15]:
import pandas as pd
import numpy as np

# Function to read and parse the external file
def read_transportation_file(file_path):
    """
    Reads a file containing the transportation problem parameters.
    The file should specify:
      - Supply array
      - Demand array
      - Cost matrix
    """
    try:
        data = pd.read_csv(file_path, delimiter=';')
        print("Data read from CSV:")
        print(data.head())
    except Exception as e:
        raise ValueError(f"Error reading the file: {e}")

    # Clean up the data by selecting only the relevant columns (first 5 columns)
    data_cleaned = data.iloc[:, :5]
    data_cleaned = data_cleaned.dropna(how='all')
    data_cleaned.reset_index(drop=True, inplace=True)

    # Extract the cost matrix (first 2 rows, first 4 columns)
    cost_matrix = data_cleaned.iloc[:2, :].values.astype(float)

    # Extract the supply (last column of the first 2 rows)
    supply = data_cleaned.iloc[:2, -1].dropna().values.astype(float)

    # Extract the demand (last row)
    demand = data_cleaned.iloc[2, :].values.astype(float)

    # Debugging output
    print("Extracted Cost Matrix:")
    print(cost_matrix)
    print("Extracted Supply:")
    print(supply)
    print("Extracted Demand:")
    print(demand)

    return supply, demand, cost_matrix




In [16]:
# Balancing the transportation problem
def balance_transportation_problem(supply, demand, cost_matrix):
    total_supply = supply.sum()
    total_demand = demand.sum()

    if total_supply > total_demand:
        demand = np.append(demand, total_supply - total_demand)
        cost_matrix = np.column_stack((cost_matrix, np.zeros(cost_matrix.shape[0])))
    elif total_demand > total_supply:
        supply = np.append(supply, total_demand - total_supply)
        cost_matrix = np.row_stack((cost_matrix, np.zeros(cost_matrix.shape[1])))

    assert cost_matrix.shape[0] == len(supply), "Supply vector does not match cost matrix rows"
    assert cost_matrix.shape[1] == len(demand), "Demand vector does not match cost matrix columns"

    return supply, demand, cost_matrix



In [17]:
# Transportation Simplex Algorithm
def transportation_simplex(cost_matrix, allocation):
    """
    Optimize the transportation problem using the simplex method.

    :param cost_matrix: The cost matrix (m x n)
    :param allocation: Initial feasible allocation (m x n)
    :return: Optimized allocation and total cost
    """
    while True:
        # Step 1: Calculate potentials (U, V)
        m, n = cost_matrix.shape
        U = np.full(m, np.nan)
        V = np.full(n, np.nan)
        U[0] = 0  # Start with the first row potential as 0

        for _ in range(m + n):
            for i in range(m):
                for j in range(n):
                    if allocation[i, j] > 0:  # Only consider occupied cells
                        if np.isnan(U[i]) and not np.isnan(V[j]):
                            U[i] = cost_matrix[i, j] - V[j]
                        elif np.isnan(V[j]) and not np.isnan(U[i]):
                            V[j] = cost_matrix[i, j] - U[i]

        # Step 2: Calculate reduced costs for non-allocated cells
        delta = np.full((m, n), np.inf)
        for i in range(m):
            for j in range(n):
                if allocation[i, j] == 0:  # Only consider empty cells
                    delta[i, j] = cost_matrix[i, j] - (U[i] + V[j])

        # Step 3: Check optimality
        min_delta = np.min(delta)
        if min_delta >= 0:
            break  # Optimal solution found

        # Step 4: Find entering variable (cell with most negative delta)
        i, j = np.unravel_index(np.argmin(delta), delta.shape)

        # Step 5: Form a cycle and adjust allocations
        path = find_cycle(allocation, (i, j))
        adjust_allocation(allocation, path)

    # Calculate total cost
    total_cost = np.sum(allocation * cost_matrix)
    return allocation, total_cost



In [18]:
# Helper function to find a cycle
def find_cycle(allocation, start):
    """Find a cycle in the allocation matrix including the starting cell."""
    m, n = allocation.shape
    visited = set()
    path = []

    def dfs(cell):
        if cell in visited:
            return True
        visited.add(cell)
        path.append(cell)
        i, j = cell
        for k in range(n):
            if allocation[i, k] > 0 and (i, k) != cell:
                if dfs((i, k)):
                    return True
        for k in range(m):
            if allocation[k, j] > 0 and (k, j) != cell:
                if dfs((k, j)):
                    return True
        path.pop()
        return False

    dfs(start)
    return path

# Helper function to adjust allocation along a cycle
def adjust_allocation(allocation, path):
    """Adjust the allocation along the cycle to improve the solution."""
    # Find the minimum allocation on the subtractive cells
    subtractive_cells = path[1::2]  # Every second cell in the cycle
    min_alloc = min(allocation[i, j] for i, j in subtractive_cells)

    # Adjust allocations along the cycle
    add = True
    for i, j in path:
        if add:
            allocation[i, j] += min_alloc
        else:
            allocation[i, j] -= min_alloc
        add = not add
        


In [19]:
# Northwest Corner Rule
def northwest_corner(supply, demand):
    supply = supply.copy()
    demand = demand.copy()
    rows, cols = len(supply), len(demand)
    allocation = np.zeros((rows, cols))

    i, j = 0, 0
    while i < rows and j < cols:
        alloc = min(supply[i], demand[j])
        allocation[i, j] = alloc
        supply[i] -= alloc
        demand[j] -= alloc
        if supply[i] == 0:
            i += 1
        if demand[j] == 0:
            j += 1
    return allocation

# Minimum Cost Method
def minimum_cost_method(supply, demand, cost_matrix):
    supply = supply.copy()
    demand = demand.copy()
    allocation = np.zeros_like(cost_matrix)
    cost_matrix = cost_matrix.copy()

    while np.any(supply > 0) and np.any(demand > 0):
        min_cost_idx = np.unravel_index(np.argmin(cost_matrix, axis=None), cost_matrix.shape)
        i, j = min_cost_idx
        alloc = min(supply[i], demand[j])
        allocation[i, j] = alloc
        supply[i] -= alloc
        demand[j] -= alloc
        cost_matrix[i, j] = np.inf  # Mark cell as used

    return allocation

# Minimum Row Cost Method
def minimum_row_cost_method(supply, demand, cost_matrix):
    supply = supply.copy()
    demand = demand.copy()
    allocation = np.zeros_like(cost_matrix)
    cost_matrix = cost_matrix.copy()

    for i in range(len(supply)):
        while supply[i] > 0:
            valid_columns = cost_matrix[i] != np.inf
            if not np.any(valid_columns):
                print(f"Row {i} is exhausted or has no valid columns left, skipping.")
                break

            valid_costs = cost_matrix[i][valid_columns]
            if valid_costs.size == 0:
                break

            j = np.argmin(valid_costs)
            j = np.where(valid_columns)[0][j]

            alloc = min(supply[i], demand[j])
            print(f"Allocating {alloc} units to cell ({i}, {j})")
            allocation[i, j] = alloc
            supply[i] -= alloc
            demand[j] -= alloc

            if demand[j] == 0:
                cost_matrix[:, j] = np.inf

    return allocation

# Vogel's Approximation Method
def vogels_method(supply, demand, cost_matrix):
    supply = supply.copy()
    demand = demand.copy()
    allocation = np.zeros_like(cost_matrix)
    cost_matrix = cost_matrix.copy()

    while np.any(supply > 0) and np.any(demand > 0):
        row_penalties = []
        for row in cost_matrix:
            valid_costs = row[row != np.inf]
            if len(valid_costs) > 1:
                row_penalties.append(np.partition(valid_costs, 1)[1] - valid_costs.min())
            else:
                row_penalties.append(0)

        col_penalties = []
        for col in cost_matrix.T:
            valid_costs = col[col != np.inf]
            if len(valid_costs) > 1:
                col_penalties.append(np.partition(valid_costs, 1)[1] - valid_costs.min())
            else:
                col_penalties.append(0)

        max_row_penalty = max(row_penalties)
        max_col_penalty = max(col_penalties)

        if max_row_penalty >= max_col_penalty:
            i = row_penalties.index(max_row_penalty)
            valid_columns = cost_matrix[i] != np.inf
            valid_costs = cost_matrix[i][valid_columns]
            if valid_costs.size == 0:
                print(f"Row {i} is exhausted, skipping.")
                continue

            j = np.argmin(valid_costs)
            j = np.where(valid_columns)[0][j]
        else:
            j = col_penalties.index(max_col_penalty)
            valid_rows = cost_matrix[:, j] != np.inf
            valid_costs = cost_matrix[:, j][valid_rows]
            if valid_costs.size == 0:
                print(f"Column {j} is exhausted, skipping.")
                continue

            i = np.argmin(valid_costs)
            i = np.where(valid_rows)[0][i]

        alloc = min(supply[i], demand[j])
        print(f"Allocating {alloc} units to cell ({i}, {j})")
        allocation[i, j] = alloc
        supply[i] -= alloc
        demand[j] -= alloc

        if supply[i] == 0:
            cost_matrix[i, :] = np.inf
        if demand[j] == 0:
            cost_matrix[:, j] = np.inf

    return allocation



In [20]:
# Example execution
def run_example(file_path):
    print("Reading the transportation problem data...")
    supply, demand, cost_matrix = read_transportation_file(file_path)

    print("\nBalancing the problem...")
    supply, demand, cost_matrix = balance_transportation_problem(supply, demand, cost_matrix)
    print("Balanced Supply:", supply)
    print("Balanced Demand:", demand)
    print("Balanced Cost Matrix:\n", cost_matrix)

    print("\nRunning Northwest Corner Rule...")
    nw_allocation = northwest_corner(supply, demand)
    print("Northwest Corner Allocation:\n", nw_allocation)

    print("\nRunning Minimum Cost Method...")
    mcm_allocation = minimum_cost_method(supply, demand, cost_matrix)
    print("Minimum Cost Allocation:\n", mcm_allocation)

    print("\nRunning Minimum Row Cost Method...")
    mrm_allocation = minimum_row_cost_method(supply, demand, cost_matrix)
    print("Minimum Row Cost Allocation:\n", mrm_allocation)

    print("\nRunning Vogel's Approximation Method...")
    vogels_allocation = vogels_method(supply, demand, cost_matrix)
    print("Vogel's Allocation:\n", vogels_allocation)

    print("\nOptimizing using Transportation Simplex Algorithm...")
    optimized_allocation, total_cost = transportation_simplex(cost_matrix, nw_allocation)
    print("Optimized Allocation:\n", optimized_allocation)
    print("Total Cost:", total_cost)

# Replace 'transportation_problem.csv' with your file path
file_path = "transportation_problem.csv"
run_example(file_path)

Reading the transportation problem data...
Data read from CSV:
      10      2     20    11     15  Unnamed: 5  Unnamed: 6  Unnamed: 7  \
0   12.0    7.0    9.0  20.0   25.0         NaN         NaN         NaN   
1    4.0   14.0   16.0  18.0   10.0         NaN         NaN         NaN   
2  100.0  150.0  200.0  50.0  500.0         NaN         NaN         NaN   
3    NaN    NaN    NaN   NaN    NaN         NaN         NaN         NaN   
4    NaN    NaN    NaN   NaN    NaN         NaN         NaN         NaN   

   Unnamed: 8  Unnamed: 9  Unnamed: 10  Unnamed: 11  Unnamed: 12  Unnamed: 13  \
0         NaN         NaN          NaN          NaN          NaN          NaN   
1         NaN         NaN          NaN          NaN          NaN          NaN   
2         NaN         NaN          NaN          NaN          NaN          NaN   
3         NaN         NaN          NaN          NaN          NaN          NaN   
4         NaN         NaN          NaN          NaN          NaN          NaN   
